In [13]:
pwd

'/home/wsuser/work'

In [14]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_a52c887f20bf41468f32fc39f1d88cd8 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='t6e9lqUJeBBJJVCNi_ZIIR7I6T56JYT2qsy6A4yuErJd',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.eu.cloud-object-storage.appdomain.cloud')

streaming_body_2 = client_a52c887f20bf41468f32fc39f1d88cd8.get_object(Bucket='plantdiseaseidentification-donotdelete-pr-ls9zwh6qazl9iv', Key='fruit-dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [15]:
ls

# Unzip the folder

In [16]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)


In [17]:
ls

fruit-dataset/


In [18]:
pwd

'/home/wsuser/work'

# Image augementation

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [22]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1)

In [23]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/fruit-dataset/train',target_size=(128,128),batch_size=32,class_mode='categorical')

Found 5384 images belonging to 6 classes.


In [24]:
x_test = test_datagen.flow_from_directory('/home/wsuser/work/fruit-dataset/test',target_size=(128,128),batch_size=32,class_mode='categorical')

Found 1686 images belonging to 6 classes.


# Import the models

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPool2D,Flatten

# Initialize the models

In [26]:
model=Sequential()

# Add CNN Layers

In [27]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [28]:
x_train.class_indices

{'Apple___Black_rot': 0,
 'Apple___healthy': 1,
 'Corn_(maize)___Northern_Leaf_Blight': 2,
 'Corn_(maize)___healthy': 3,
 'Peach___Bacterial_spot': 4,
 'Peach___healthy': 5}

# Add Pooling layer

In [29]:
model.add(MaxPool2D(pool_size=(2,2)))

# Add Flatten Layer

In [30]:
model.add(Flatten())

# Add Dense Layer

In [31]:
model.add(Dense(40, kernel_initializer='uniform',activation='relu'))
model.add(Dense(20, kernel_initializer='random_uniform',activation='relu'))

# Add Output layer

In [32]:
model.add(Dense(6,activation='softmax', kernel_initializer='random_uniform'))

# Compile the model

In [33]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Fit the model

In [34]:
model.fit_generator(x_train,steps_per_epoch=168,validation_data=x_test,validation_steps=52,epochs=3)

/tmp/wsuser/ipykernel_164/2881909211.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(x_train,steps_per_epoch=168,validation_data=x_test,validation_steps=52,epochs=3)


Epoch 1/3
168/168 [==============================] - 83s 491ms/step - loss: 0.9961 - accuracy: 0.6371 - val_loss: 185.4097 - val_accuracy: 0.6010
Epoch 2/3
168/168 [==============================] - 82s 487ms/step - loss: 0.4763 - accuracy: 0.8272 - val_loss: 265.1531 - val_accuracy: 0.5925
Epoch 3/3
168/168 [==============================] - 81s 484ms/step - loss: 0.3537 - accuracy: 0.8757 - val_loss: 239.7596 - val_accuracy: 0.6785


# save the model

In [35]:
model.save("fruit.h5")

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 63, 63, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 127008)            0         
                                                                 
 dense (Dense)               (None, 40)                5080360   
                                                                 
 dense_1 (Dense)             (None, 20)                820       
                                                                 
 dense_2 (Dense)             (None, 6)                 126       
                                                        

# Test the model

In [37]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np

In [38]:
model = load_model("fruit.h5")

# Test Apple_Healthy Class images


In [39]:
pwd

'/home/wsuser/work'

In [40]:
img = image.load_img('/home/wsuser/work/fruit-dataset/test/Apple___healthy/00fca0da-2db3-481b-b98a-9b67bb7b105c___RS_HL 7708.JPG',target_size=(128,128))

In [41]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)

In [46]:
#pred = model.predict_classes(x)
pred = np.argmax(model.predict(x),axis=1)

In [47]:
print(pred)


[1]


In [48]:
index =['Apple___Black_rot','Apple___healthy','Corn_(maize)___Northern_Leaf_Blight','Corn_(maize)___healthy','Peach___Bacterial_spot','Peach___healthy']

In [49]:
print('the given image belogs to=',index[pred[0]])

the given image belogs to= Apple___healthy


# Test Apple Black Rot class images

In [50]:
img = image.load_img('/home/wsuser/work/fruit-dataset/test/Apple___Black_rot/0f3d45f4-e121-42cd-a5b6-be2f866a0574___JR_FrgE.S 2870.JPG',target_size=(128,128))

In [51]:
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred = np.argmax(model.predict(x),axis=1)
print('the given image belogs to=',index[pred[0]])

the given image belogs to= Apple___Black_rot
